# 1. Setup

Import libraries

In [ ]:

!pip install torch
!pip install transformers













     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 45.9 MB/s eta 0:00:00




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.8 MB/s eta 0:00:00








     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 33.5 MB/s eta 0:00:00



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.7 MB/s eta 0:00:00






     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 12.3 MB/s eta 0:00:00








In [ ]:
import pandas as pd
import json
import random,operator
import numpy as np
import os, re, string
import torch
import re
from sklearn.decomposition import TruncatedSVD
import torch.nn as nn
import logging
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW, get_linear_schedule_with_warmup, RobertaModel,PhobertTokenizer

In [ ]:
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
!nvcc --version

PyTorch version: 2.1.0+cu118

CUDA version: 11.8

nvcc: NVIDIA (R) Cuda compiler driver

Copyright (c) 2005-2022 NVIDIA Corporation

Built on Wed_Sep_21_10:33:58_PDT_2022

Cuda compilation tools, release 11.8, V11.8.89

Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
seed = 42
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)
!nvidia-smi

Device:  cpu

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
# connect gg drive
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/Colab Notebooks/Challenge'

Mounted at /content/gdrive

/content/gdrive/MyDrive/Colab Notebooks/Challenge


# 2. Prepare model, data

In [ ]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=device, abbreviated=False)

ValueError: ignored

### 2.0. Load Data

In [ ]:
# Phobert token
train_input_ids = torch.load("train_input_ids.pt")
train_attention_mask = torch.load("train_attention_mask.pt")
train_labels = torch.load("train_labels.pt")

test_input_ids = torch.load("test_input_ids.pt")
test_attention_mask = torch.load("test_attention_mask.pt")

In [ ]:
# Svd matrix
train_svd_matrix = torch.load("train_svd_matrix.pt")
test_svd_matrix = torch.load("test_svd_matrix.pt")

In [ ]:
# Phobert token
train_input_ids=train_input_ids.cpu().numpy()
train_attention_mask=train_attention_mask.cpu().numpy()
test_input_ids=test_input_ids.cpu().numpy()
test_attention_mask=test_attention_mask.cpu().numpy()
# Svd matrix
train_svd_matrix=train_svd_matrix.cpu().numpy()
test_svd_matrix=test_svd_matrix.cpu().numpy()

In [ ]:
# Phobert token
train_input_ids = torch.from_numpy(train_input_ids)
train_attention_mask = torch.from_numpy(train_attention_mask)
test_input_ids = torch.from_numpy(test_input_ids)
test_attention_mask = torch.from_numpy(test_attention_mask)
# Svd matrix
train_svd_matrix = torch.from_numpy(train_svd_matrix)
test_svd_matrix = torch.from_numpy(test_svd_matrix)

In [ ]:
print(train_input_ids.shape)
print(train_attention_mask.shape)
print(test_input_ids.shape)
print(test_attention_mask.shape)
print(train_svd_matrix.shape)
print(test_svd_matrix.shape)

In [ ]:
# Phobert token
# train_input_ids = train_input_ids.to(device)
# train_attention_mask = train_attention_mask.to(device)
# test_input_ids = train_input_ids.to(device)
# test_attention_mask = train_attention_mask.to(device)
# Svd matrix
# train_svd_matrix = train_svd_matrix.to(device)
# test_svd_matrix = test_svd_matrix.to(device)

### 2.1. Fine-tuning Phobert Model

In [ ]:
# Phobert model
num_labels = 3
# tokenizer = PhobertTokenizer.from_pretrained('vinai/phobert-base')
phobert_model = RobertaModel.from_pretrained('vinai/phobert-base', num_labels=num_labels).to(device)

# model = CNNModel(num_labels,phobert_model ).to(device)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.ERROR)
class MyPhoBERTModel(nn.Module):
    def __init__(self, num_labels,phobert_model):
        super(MyPhoBERTModel, self).__init__()
        self.phobert = phobert_model
        self.reduce_mean_layers = nn.Sequential(
            nn.LayerNorm(768),
            nn.Linear(768, 768),
            nn.LayerNorm(768),
            nn.Linear(768, 768),
            nn.LayerNorm(768),
            nn.Linear(768, 768)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.phobert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_states = outputs.last_hidden_state

        # Apply reduce_mean on the last three layers' hidden states
        reduced_hidden_states = self.reduce_mean_layers(last_hidden_states[:, -4:, :]).mean(dim=1)

        return {
            'last_hidden_state': reduced_hidden_states
        }

# Example usage

phobert_model = MyPhoBERTModel(num_labels,phobert_model).to(device)
# input_ids = tokenizer.encode("Hello, how are you?", add_special_tokens=True, truncation=True, padding='max_length', max_length=64, return_tensors='pt')
# attention_mask = input_ids.gt(0)


# outputs = phobert_model(train_input_ids, train_attention_mask)
# train_phobert_embeddings = outputs['last_hidden_state']

In [ ]:
batch_size = 10  # Giá trị batch size nhỏ hơn

num_samples = train_input_ids.size(0)  # Số lượng mẫu huấn luyện
num_batches = num_samples // batch_size  # Số lượng batch

train_phobert_embeddings = []  # Danh sách để tích lũy kết quả embeddings

for i in range(num_batches):
    start_index = i * batch_size
    end_index = (i + 1) * batch_size

    batch_input_ids = train_input_ids[start_index:end_index]
    batch_attention_mask = train_attention_mask[start_index:end_index]

    # Chuyển batch sang GPU (nếu cần)
    batch_input_ids = batch_input_ids.to(device)
    batch_attention_mask = batch_attention_mask.to(device)

    # Truyền batch vào mô hình
    outputs = phobert_model(batch_input_ids, batch_attention_mask)
    batch_embeddings = outputs['last_hidden_state']

    # Gộp batch_embeddings vào danh sách tổng
    train_phobert_embeddings.append(batch_embeddings)

# Xử lý batch cuối cùng (nếu có)
if num_samples % batch_size != 0:
    start_index = num_batches * batch_size
    end_index = num_samples

    batch_input_ids = train_input_ids[start_index:end_index]
    batch_attention_mask = train_attention_mask[start_index:end_index]

    # Chuyển batch sang GPU (nếu cần)
    batch_input_ids = batch_input_ids.to(device)
    batch_attention_mask = batch_attention_mask.to(device)

    # Truyền batch vào mô hình
    outputs = phobert_model(batch_input_ids, batch_attention_mask)
    batch_embeddings = outputs['last_hidden_state']

    # Gộp batch_embeddings vào danh sách tổng
    train_phobert_embeddings.append(batch_embeddings)

# Tạo tensor từ danh sách embeddings


In [ ]:
train_phobert_embeddings = torch.cat(train_phobert_embeddings, dim=0)

### 2.2. CNN model

In [ ]:
# Combine PhoBERT embeddings and SVD embeddings
# combined_embeddings = torch.cat((phobert_embeddings, torch.tensor(svd_matrix)), dim=1)

# Define CNN model
class CNNClassifier(nn.Module):
    def __init__(self, input_dim, num_filters, kernel_sizes, num_classes):
        super(CNNClassifier, self).__init__()
        self.convs = nn.ModuleList([nn.Conv1d(in_channels=input_dim, out_channels=num_filters, kernel_size=ks) for ks in kernel_sizes])
        self.fc = nn.Linear(len(kernel_sizes) * num_filters, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension (batch_size, channels, seq_len)
        x = [F.relu(conv(x)) for conv in self.convs]  # Apply convolution and ReLU activation
        x = [F.max_pool1d(c, c.size(2)).squeeze(2) for c in x]  # Max pooling
        x = torch.cat(x, 1)  # Concatenate pooled features from different kernel sizes
        x = F.dropout(x, p=0.5, training=self.training)  # Apply dropout
        x = self.fc(x)  # Fully connected layer
        return x

# Initialize CNN model
# input_dim = combined_embeddings.size(1)
num_filters = 100
kernel_sizes = [3, 4, 5]
num_classes = 3  # Number of verdict classes
model = CNNClassifier(1024, num_filters, kernel_sizes, num_classes)

# # Forward pass
# outputs = model(combined_embeddings)

# print(outputs)

In [ ]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra

# Chia tập train thành tập train và tập validation
train_input_ids, val_input_ids, train_attention_mask, val_attention_mask, train_labels, val_labels =train_test_split(train_input_ids, train_attention_mask, train_labels, test_size=0.2, random_state=seed)

# Initialization

In [ ]:
max_length = 10
# Các tham số huấn luyện
batch_size = 10
epochs = 5
learning_rate = 2e-5
shuffle = True
num_workers = 2
# Tạo train dataset từ tập train
train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)

# Tạo validation dataset từ tập validation
val_dataset = TensorDataset(val_input_ids, val_attention_mask, val_labels)

def custom_collate(batch):
    # Trích xuất các thành phần từ batch
    input_ids_batch, attention_mask_batch, labels_batch = zip(*batch)

    # Đảm bảo các dữ liệu có cùng kích thước
    max_length = max(len(ids) for ids in input_ids_batch)
    input_ids_batch = [torch.cat((ids, torch.tensor([1] * (max_length - len(ids)), dtype=torch.long))) for ids in input_ids_batch]
    attention_mask_batch = [torch.cat((mask, torch.tensor([1] * (max_length - len(mask)), dtype=torch.long))) for mask in attention_mask_batch]

    # Chuyển đổi thành tensor
    input_ids_batch = torch.stack(input_ids_batch)
    attention_mask_batch = torch.stack(attention_mask_batch)
    labels_batch = torch.tensor(labels_batch, dtype=torch.long)

    return input_ids_batch, attention_mask_batch, labels_batch

# Tạo train dataloader từ train dataset và custom collate function
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate, num_workers=num_workers, worker_init_fn=np.random.seed(seed))

# Tạo validation dataloader từ validation dataset và custom collate function
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=custom_collate, shuffle=False, num_workers=num_workers )

In [ ]:


# Tạo optimizer và scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning

  warnings.warn(


# Train and Evaluation

In [ ]:
# Huấn luyện mô hình
# Tính toán độ dài câu tối đa trong batch



model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in train_dataloader:
        input_ids = batch[0].to(torch.long)

        attention_mask = batch[1].to(torch.long)
        labels = batch[2].to(torch.long)
        input_ids= input_ids.to(device)
        attention_mask= attention_mask.to(device)
        labels= labels.to(device)

        optimizer.zero_grad()

        logits = model(input_ids, attention_mask)
        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(logits, labels)
        # outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        # loss = outputs.loss

        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{epochs} - Average Loss: {average_loss}")


In [ ]:
# Đánh giá mô hình trên tập kiểm tra
model.eval()
predictions = []
with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch[0].to(torch.long)
        attention_mask = batch[1].to(torch.long)
        labels = batch[2].to(torch.long)
        input_ids= input_ids.to(device)
        attention_mask= attention_mask.to(device)
        labels= labels.to(device)

        logits = model(input_ids, attention_mask)
        _, predicted_classes = torch.max(logits, dim=1)

        predictions.extend(predicted_classes.cpu().numpy())


In [ ]:

# Tính toán độ chính xác và các độ đo đánh giá khác
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(val_labels, predictions)
precision = precision_score(val_labels, predictions, average="weighted")
recall = recall_score(val_labels, predictions, average="weighted")
f1 = f1_score(val_labels, predictions, average="weighted")

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

Accuracy: 0.768075639599555

Precision: 0.7740248049812857

Recall: 0.768075639599555

F1-Score: 0.767352098980306


# Prediction Labels

In [ ]:
test_input_ids = torch.load("test_input_ids.pt")
test_attention_mask = torch.load("test_attention_mask.pt")

In [ ]:
test_input_ids=test_input_ids.cpu().numpy()
test_attention_mask=test_attention_mask.cpu().numpy()

In [ ]:
test_input_ids = torch.from_numpy(test_input_ids)
test_attention_mask = torch.from_numpy(test_attention_mask)

In [ ]:
test_dataset = TensorDataset(test_input_ids, test_attention_mask)

# Tạo DataLoader

test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, worker_init_fn=np.random.seed(seed))

In [ ]:
val_labels

tensor([0, 2, 1,  ..., 1, 1, 2])

In [ ]:

# Chạy mô hình trên dữ liệu kiểm tra
model.eval()  # Chuyển sang chế độ đánh giá
predictions_test = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch[0].to(torch.long)
        attention_mask = batch[1].to(torch.long)
        input_ids= input_ids.to(device)
        attention_mask= attention_mask.to(device)

        logits = model(input_ids, attention_mask)
        _, predicted_labels = torch.max(logits, 1)
        predictions_test.extend(predicted_labels.cpu().numpy())


In [ ]:
predicted_labels = [prediction for prediction in predictions_test]

In [ ]:

predicted_labels  = torch.tensor(predicted_labels)
predicted_labels

tensor([1, 0, 2,  ..., 1, 0, 2])

In [ ]:
day = 20
torch.save(predicted_labels, day+"_predicted_verdicts_labels.pt")